<img align="right" width="250" height="150" src="https://lh3.googleusercontent.com/p/AF1QipPWZQfa087JiVjutpUTVEGRnh6W214Wjm439gKQ=w1080-h608-p-no-v0">

## **Proyecto:** Retención de usuarios de la Plataforma Digital

#### El objetivo de este notebook es realizar el análisis EDA del archivo de `..\Turnero_YYYYMM.parquet` con el objeto de ver que datos incorporar en el archivo de `..\Crosseling_ampliado_YYYYMM.parquet`.

In [ ]:
import pandas as pd
import pandas_profiling
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

try:
    # Monto la unidad de drive porque los archivos son muy grandes
    from google.colab import drive
    drive.mount('/content/drive')
    DIR = '/content/drive/MyDrive/Trayecto3-DataDriven/Data/'
    # En GOOGLE COLAB hace falta instalar la última versión de pandas profiling
    ! pip install --use-feature=2020-resolver https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
except:
    # Sino lo consigo asumo que estoy trabajando local
    DIR = os.path.abspath(r'..\Data\Export') + '\\'

In [ ]:
uc = [
    'Proceso',
    'Tipo_atencion',
    'Fecha_creada',
    'Fecha_turno',
    'Estado',
    'Hora_desde',
    'Hora_hasta'
]

In [ ]:
file = DIR + 'Turnero.csv'
df = pd.read_csv(file, index_col=False, columns=uc)

In [ ]:
df.info()

In [ ]:
# Analizamos
df.reset_index(drop=True, inplace=True)
report = pandas_profiling.ProfileReport(df)
report

In [ ]:
# Indexamos por Fecha turno
df.set_index('Fecha_turno', inplace=True)


In [ ]:
# Filtramos para este un mes a ver que pasa
df = df[(df.index >= datetime(2022, 1, 1)) & ((df.index < datetime(2022, 5, 1)))]
df

In [ ]:
# Analizamos nuevamente
report = pandas_profiling.ProfileReport(df)
report

In [ ]:
# Filtramos para los turnos concretados
df = df[df['Estado']=='FINALIZADA'].drop('Estado', axis=1)

In [ ]:
df

In [ ]:
# Analizamos nuevamente
report = pandas_profiling.ProfileReport(df)
report

In [ ]:
plt.figure(figsize=(16, 9))
sns.lineplot(data = df['Proceso'].groupby(['Fecha_turno']).aggregate('count'))

In [ ]:
for caracteristica in ['Proceso', 'Tipo_atencion']:
    df_tn_cnt = df[[caracteristica]].pivot_table(index='Fecha_turno', columns=caracteristica, aggfunc=len, fill_value=0)
    df_tn_cnt.plot(figsize=(16, 9))